Import all necessary libraries

In [1]:
import glob
import pandas as pd
import sqlite3
from currency_converter import CurrencyConverter

Read all data from folder to dataframe

In [2]:
path = 'itunes_dataset'
all_files = glob.glob(path + "/*.txt")
li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0, sep='\t')
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True, sort=False)

Replace all spaces in column names and convert Date to datatime

In [3]:
frame.columns = [column.replace(' ', '_') for column in frame.columns]
frame['Event_Date'] =  pd.to_datetime(frame['Event_Date'], format='%Y-%m-%d')

Create connection to Sqlite database

In [4]:
cnx = sqlite3.connect(':memory:')
db = sqlite3.connect('itunes_data.db')

Separete App data and save it to App table.

In [5]:
App_columns = ['App_Name', 'App_Apple_ID']
App=frame[App_columns].drop_duplicates().reset_index(drop=True)
App.head()
App.to_sql('App', db, if_exists='append', index=False)

Separete Subscription data and save it to Subscription table.

In [6]:
subs_columns = ["Subscription_Name", "Subscription_Apple_ID", "Subscription_Group_ID", "Subscription_Duration"]
Subscription=frame[subs_columns].drop_duplicates().reset_index(drop=True)
Subscription.to_sql('Subscription', db, if_exists='append', index=False)

Separete Countries data and save it to Countries table.

In [7]:
country_columns = ["Country", "Customer_Currency"]
Countries=frame[country_columns].drop_duplicates().reset_index(drop=True)
Countries.to_sql('Countries', db, if_exists='append', index=False)

Convert all Customer_Price to prices in USD.

In [8]:
cc = CurrencyConverter('currencies.csv')

def convert_to_dollars(row):
    return cc.convert(row['Customer_Price'], row['Customer_Currency'], 'USD')

frame['USD_Price'] = frame.apply(convert_to_dollars, axis=1)

Separete all remaining columns and foreign keys and save them in Events table.

In [9]:
columns = ['App_Name',
 'Subscription_Name',
 'Subscription_Group_ID',
 'Subscription_Duration',
 'Customer_Currency']
event_columns = [column for column in frame.columns if column not in columns]

In [10]:
Event=frame[event_columns].drop_duplicates().reset_index(drop=True)
Event.to_sql('Events', db, if_exists='append', index=False)